In [6]:
import os
import shutil
import unittest
from catvae.trainer import LightningCatVAE, LightningLinearVAE
from catvae.sim import multinomial_bioms
from biom import Table
from biom.util import biom_open
import numpy as np
from pytorch_lightning import Trainer
import argparse
import seaborn as sns

In [2]:
import torch
torch.__version__

'1.6.0'

# Simulate data

In [3]:
np.random.seed(0)
k = 10
sims = multinomial_bioms(k=k, D=50, N=250, M=100000)
Y = sims['Y']
parts = Y.shape[0] // 10
samp_ids = list(map(str, range(Y.shape[0])))
obs_ids = list(map(str, range(Y.shape[1])))
train = Table(Y[:parts * 8].T, obs_ids, samp_ids[:parts * 8])
test = Table(Y[parts * 8 : parts * 9].T,
             obs_ids, samp_ids[parts * 8 : parts * 9])
valid = Table(Y[parts * 9:].T, obs_ids, samp_ids[parts * 9:])
tree = sims
with biom_open('train.biom', 'w') as f:
    train.to_hdf5(f, 'train')
with biom_open('test.biom', 'w') as f:
    test.to_hdf5(f, 'test')
with biom_open('valid.biom', 'w') as f:
    valid.to_hdf5(f, 'valid')

In [4]:
sims['tree'].write('basis.nwk')

'basis.nwk'

# Train model

In [6]:
output_dir = 'output'
args = [
    '--train-biom', 'train.biom',
    '--test-biom', 'train.biom',
    '--val-biom', 'train.biom',
    '--basis', 'basis.nwk',
    '--output-directory', output_dir,
    '--epochs', '5000',
    '--batch-size', '200',
    '--num-workers', '10',
    '--scheduler', 'steplr',
    '--likelihood', 'multinomial',
    '--use-analytic-elbo', 'False',
    '--learning-rate', '1e-3',
    '--n-latent', str(k),
    '--gpus', '1'
]
parser = argparse.ArgumentParser(add_help=False)
#parser = LightningCatVAE.add_model_specific_args(parser)
parser = LightningLinearVAE.add_model_specific_args(parser)
parser.add_argument('--num-workers', type=int)
parser.add_argument('--gpus', type=int)
args = parser.parse_args(args)
model = LightningLinearVAE(args)
model.set_eigs(sims['eigvectors'], sims['eigs'])

trainer = Trainer(
    max_epochs=args.epochs,
    gpus=args.gpus,
    check_val_every_n_epoch=10,
    # profiler=profiler,
    fast_dev_run=False,
    # auto_scale_batch_size='power'
)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | LinearVAE | 991   


Exception ignored in: <function _releaseLock at 0x7ff279778ee0>
Traceback (most recent call last):
  File "/home/juermieboop/miniconda3/envs/catvae/lib/python3.8/logging/__init__.py", line 223, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


RuntimeError: DataLoader worker (pid(s) 68956, 68957, 68958, 68959, 68960) exited unexpectedly

In [3]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
%tensorboard --logdir lightning_logs

Reusing TensorBoard on port 6006 (pid 20834), started 4:19:46 ago. (Use '!kill 20834' to kill it.)

In [ ]:
trainer.model

# Evaluate the model

In [3]:
!ls lightning_logs/version_73/checkpoints

'epoch=19.ckpt'


In [5]:
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
checkpoint_dir = 'lightning_logs/version_73/checkpoints'
path = f'{checkpoint_dir}/epoch=19.ckpt'
model = LightningCatVAE.load_from_checkpoint(path).cuda()

NameError: name 'LightningCatVAE' is not defined

In [ ]:
#W = model.model.get_loadings()
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist

W = model.model.decoder.weight.detach().cpu().numpy()

d_estW = pdist(W)
dW = pdist(sims['W'])
# we appear to be off by a factor of 2 -- where is this coming from?
plt.scatter(dW, d_estW)
plt.plot(np.linspace(0, 4), np.linspace(0, 4), 'r')
plt.xlabel('Predicted correlations')
plt.ylabel('Actual correlations')

print(pearsonr(dW, d_estW))

In [ ]:
np.diag(sims['W'].T @ sims['W'])

In [ ]:
np.diag(W.T @ W)

In [ ]:
eigvals = (W**2).sum(axis=0)
Weig = W / np.sqrt(eigvals)

In [ ]:
np.round(Weig.T @ Weig, 2)

In [ ]:
Wu, Ws, Wv = np.linalg.svd(W)
k = W.shape[1]
Wu = Wu[:, :k]
Ws = Ws[:k]
Wv = Wv[:k, :]

In [ ]:
np.linalg.norm(Weig.T @ Weig - np.eye(Weig.shape[-1]))

In [ ]:
Ws

In [ ]:
eigvals = np.sqrt(np.sort(eigvals)[::-1])

In [ ]:
np.sqrt(np.sum((eigvals - Ws)**2))

In [ ]:
sims['sigma']

In [ ]:
torch.exp(model.model.log_sigma_sq)

In [ ]:
model.model.log_sigma_sq